# Lab 5: Land Surface Temperature using CORINE-based Emissivity

## 🎯 Objectives
In this exercise, you will:
- Select a cloud-free Landsat 8 images from 2013 and 2023 (or different if you're sure that you'll spot a difference in land cover)
- Calculate Brightness Temperature (TB) from Band 10.
- Load CORINE Land Cover data and assign emissivity values to each land cover class.
- Use the Planck-based formula to calculate Land Surface Temperature (LST).
- Visualize and interpret the results.

## Step 1: Define Area of Interest (AOI)
- Use coordinates around Reduta street in Kraków.
- Create a polygon or rectangle using `ee.Geometry.Polygon`.

In [1]:
import ee
import geemap
# ee.Authenticate()
ee.Initialize(project="ee-zuzannaslobodzian2")

In [2]:
# Define AOI here
aoi = ee.Geometry.Rectangle([19.884968, 50.025676, 19.908652, 50.012329])

## Step 2: Load Landsat 8 imagery for the dates you've picked
- Filter for low cloud cover (< 20%)
- Select Band 10 and convert to TB using: `TB = ST_B10 * 0.00341802 + 149.0`

In [3]:
# Load image and calculate TB
sentinel_2013 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(aoi)
    .filterDate('2013-07-01', '2013-07-31')
    .filterMetadata('CLOUD_COVER', 'less_than', 20)
    .first()
)

sentinel_2024 = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterBounds(aoi)
    .filterDate('2024-08-01', '2024-08-31')
    .filterMetadata('CLOUD_COVER', 'less_than', 20)
    .first()
)

tb_image_2013 = sentinel_2013.select('ST_B10').multiply(0.00341802).add(149.0).rename('TB_2013');

tb_image_2024 = sentinel_2024.select('ST_B10').multiply(0.00341802).add(149.0).rename('TB_2024');



## Step 3: Load CORINE Land Cover data
- Use dataset `COPERNICUS/CORINE/V20/100m/2018`
- Clip it to your AOI

In [4]:
# Load CORINE
corine_2018 = (
    ee.Image('COPERNICUS/CORINE/V20/100m/2018')
)

corine_2018 = corine_2018.clip(aoi)



## Step 4: Assign emissivity to CORINE classes
- Use a dictionary for classes
- Use `remap()` and optionally a default value

In [5]:
# Create emissivity image

emissivity_dict = {
    111: 0.92,  # Continuous urban fabric
    112: 0.92,  # Discontinuous urban fabric
    121: 0.91,  # Industrial or commercial units
    211: 0.96,  # Non-irrigated arable land
    311: 0.98,  # Forests
    412: 0.97,  # Peat bogs
    324: 0.96,  # Transitional woodland-shrub
    231: 0.97   # Pastures
}

key_emissivity = list(emissivity_dict.keys())
values_emissivity = list(emissivity_dict.values())

emissivity_image = corine_2018.remap(key_emissivity, values_emissivity)

## Step 5: Calculate LST using the formula:
$$
LST = \frac{T_B}{1 + \left( \frac{\lambda \cdot T_B}{c_2} \right) \cdot \ln(\varepsilon)}
$$
- λ = 10.8 µm
- c₂ = 14388 µm·K

In [6]:
# Calculate LST

lambda_ = 10.8
c2 = 14388

lst_image = tb_image_2013.expression(
    'TB / (1 + ((lambda * TB) / c2) * log(1.0 / emissivity))', {
        'TB': tb_image_2013,
        'emissivity': emissivity_image,
        'lambda': lambda_,
        'c2': c2
    }).rename('LST')

## Step 6: Visualize the LST result
- Use palette: `['blue', 'yellow', 'red']`
- Suggested range: `min=290`, `max=325`

In [7]:
# Visualize LST

Map = geemap.Map()
Map.centerObject(aoi, zoom=15)

vis_params = {
    'palette': ['blue', 'yellow', 'red'],
    'min': 290,
    'max': 325,
}

Map.addLayer(lst_image, vis_params, 'LST')
Map

Map(center=[50.01900279353348, 19.896809999996524], controls=(WidgetControl(options=['position', 'transparent_…

## Step 7: (Optional) Analyze statistics by land cover class

In [ ]:
# Optionally compute zonal statistics

## Step 8: (Optional - Easter Egg :)) Generate your own Land Cover Classification using TerraTorch and foundation models*

Based on the example/tutorial: https://aiforgood.itu.int/event/workshop-earth-observation-foundation-models-with-prithvi-eo-2-0-and-terratorch/

*to earn 5.0 grade that will make a great impact on your final grade